In [2]:
# для автозагрузки изменений в модулях без необходимости перезагрузки kelner
%load_ext autoreload
%autoreload 2

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# необходимо для корректного импорта своих модулей в JupyterLab
import sys
current_dir = sys.path[0]
project_dir = 'SF_DS_Pro'
last_position = current_dir.find(project_dir) + len(project_dir)
project_path = current_dir[:last_position]
sys.path.append(project_path)

import Handlers as hd

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn import set_config
set_config(transform_output = 'pandas')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# sns.set_theme('notebook')
# sns.set_palette('Set2')

plt.rcParams['figure.figsize'] = (12, 8)

In [4]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [5]:
df = pd.read_csv('data/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981756 entries, 0 to 981755
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   book_id  981756 non-null  int64
 1   user_id  981756 non-null  int64
 2   rating   981756 non-null  int64
dtypes: int64(3)
memory usage: 22.5 MB


In [6]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
print(f"В обучающей выборке {train.shape[0]} объектов")

В обучающей выборке 785404 объектов


Предварительно вычислим количество уникальных пользователей и книг:

In [7]:
n_books = df.book_id.nunique()
print('Уникальных книг:', n_books)

Уникальных книг: 10000


In [8]:
n_users = df.user_id.nunique()
print('Уникальных пользователей:', n_users)

Уникальных пользователей: 53424


В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:

In [9]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

Сначала мы задаём размерность входного слоя. После этого определяем размер эмбеддинга — в данном случае снижаем размерность до 5. Далее мы разворачиваем результат в массив с одним измерением с помощью слоя Flatten().

Делаем то же самое для пользователей:

In [10]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

Размерность входного словя всегда равна длине вектора+1

Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:

In [11]:
conc = Concatenate()([book_vec, user_vec])

Далее начинаем «собирать» нашу нейронную сеть из слоёв. 

Dense обозначает полносвязный слой. 

Также мы обозначаем для него количество нейронов и данные, которые идут на вход.

In [12]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:

In [13]:
model2 = Model([user_input, book_input], out)

Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. 

В данном случае будем использовать метод adam (вариация градиентного спуска) и хорошо известную вам среднеквадратичную ошибку:

In [14]:
model2.compile(optimizer='adam', loss='mean_squared_error')

Теперь будем обучать нашу модель:

In [15]:
# history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

В параметр эпох передаём значение 5: у нас будет реализовано пять эпох — пять обучений нейронной сети.

На каждой из эпох обновляются веса для минимизации ошибки.

Теперь можно оценить качество:

In [16]:
# model2.evaluate([test.user_id, test.book_id], test.rating)

Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав десять эпох:

In [17]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 [==============================] - 82s 3ms/step - loss: 0.7993
Epoch 2/10
24544/24544 [==============================] - 83s 3ms/step - loss: 0.6883
Epoch 3/10
24544/24544 [==============================] - 83s 3ms/step - loss: 0.6623
Epoch 4/10
24544/24544 [==============================] - 76s 3ms/step - loss: 0.6393
Epoch 5/10
24544/24544 [==============================] - 76s 3ms/step - loss: 0.6194
Epoch 6/10
24544/24544 [==============================] - 76s 3ms/step - loss: 0.5997
Epoch 7/10
24544/24544 [==============================] - 88s 4ms/step - loss: 0.5792
Epoch 8/10
24544/24544 [==============================] - 86s 3ms/step - loss: 0.5606
Epoch 9/10
24544/24544 [==============================] - 88s 4ms/step - loss: 0.5447
Epoch 10/10
6136/6136 [==============================] - 6s 886us/step - loss: 0.7453


0.7452942728996277